In [1]:
# face size statistic - YRKKH 2400/average in all frames of close shot
# face num statistic - YRKKH
# overlap statistic - YRKKH 2480/average in all frames of close shot
# transition statistic

import numpy as np
import cv2
from PIL import Image
import os

root_dir = "/internal-demo3/bingyu/"
video_name = "YRKKH_3963--20230129192718416--PG--2012/"
blurred_frame_dir = root_dir+"frames/YRKKH_3963--20230129192718416--PG--2012/"

In [2]:
# get video names
video_names = [dI for dI in os.listdir('D:/videocodec_source_video/raw_frames/')]
print(video_names)

FileNotFoundError: [Errno 2] No such file or directory: 'D:/videocodec_source_video/raw_frames/'

In [5]:
# reorginize face detection results
video_names = ['YRKKH_2800_5min_360p', 'SS_clip_360p'] #'YRKKH_2699--20180722204822056--PG--2012', 'YHM_1530--20180724151448127--PG--2012', 'ISHQBAAZ_592--20180724214511411--PG--2012', 'KKBW_93--20180724010609002--PG--2012', 'YRKKH_2698--20180722045900125--PG--2012']
import pickle
from operator import itemgetter
face_det_result_dir = '/large/aws/youqiang/ml-based-encoding-improvement/testsets/face_result.txt'
for i in range(len(video_names)):
    video_name = video_names[i]
    video_det_result = []
    with open(face_det_result_dir, 'r') as f:
        det_result = f.readline()
        while det_result:
            if video_name in det_result:
                det_result = det_result.split('.png')
                frame_id = int(det_result[0].split('/')[-1])
                det_face = det_result[1].split('] [')
                bbox = eval(det_face[0]+']')
                scores = eval('['+det_face[1])
                video_det_result.append([frame_id, bbox, scores])
            det_result = f.readline()
    video_det_result = sorted(video_det_result, key=itemgetter(0))
    # print(video_det_result)
    # break
    with open(os.path.join('/home/bingyu/bingyu/',video_name+'.pkl'), 'wb') as f:
        pickle.dump(video_det_result, f)
    # with open(os.path.join('D:/facedet/',video_name+'.pkl'), 'rb') as f:
    #     temp = pickle.load(f)
    #     print(temp[0])
    # break

In [2]:
# frames that need to be blurred
video_names = ['NUKSHSB_162--20230128192154349--PG--2012']#, 'SS_clip_360p'] #'YRKKH_2699--20180722204822056--PG--2012', 'YHM_1530--20180724151448127--PG--2012', 'ISHQBAAZ_592--20180724214511411--PG--2012', 'KKBW_93--20180724010609002--PG--2012', 'YRKKH_2698--20180722045900125--PG--2012']

def get_noblur_frame(video_name):
    shot_cls_dir = root_dir + "shot_cls_result/"+video_name+"_scale.txt"
    shot_dir = root_dir + "shot_cls_result/"+video_name+"_0.1.txt"
    
    f_cls = open(shot_cls_dir)
    cls_label = f_cls.readline()
    labels = []
    frame_ind = []
    while cls_label:
        labels.append(int(cls_label.split(',')[0]))
        cls_label = f_cls.readline()
    f_cls.close()
    
    f_shot = open(shot_dir, encoding='utf-8')
    shot_ = f_shot.readline()
    for i in range(len(labels)):
        if labels[i] != 0:
            shot_ = shot_.strip('\n')
            shot_list = shot_.split('\t')
            frame_range = np.arange(int(shot_list[0])+1, int(shot_list[1])+2)
            frame_ind.extend(list(frame_range))
        shot_ = f_shot.readline()
    f_shot.close()
    
    return frame_ind

video_noblur_frames = []
for video_name in video_names:
    noblur_frame = get_noblur_frame(video_name)
    video_noblur_frames.append(noblur_frame)
    
print(len(video_noblur_frames[0]))

2804


In [15]:
# face related statistic
import pickle
import sys
# savedStdout = sys.stdout
video_names = ['YRKKH_2800_5min']#, 'SS_clip'] #'YRKKH_2699--20180722204822056--PG--2012', 'YHM_1530--20180724151448127--PG--2012', 'ISHQBAAZ_592--20180724214511411--PG--2012', 'KKBW_93--20180724010609002--PG--2012', 'YRKKH_2698--20180722045900125--PG--2012']
bbox_conf_thr = 0.9

def get_video_frame_num(video_names):
    video_frame_nums = []
    for video_name in video_names:
        frame_dir = os.path.join('/home/bingyu/testsets/frames/', video_name)
        video_frame_nums.append(len(os.listdir(frame_dir)))
    return video_frame_nums

video_frame_nums = get_video_frame_num(video_names)
bbox_scale = 1080/360
for i in range(len(video_names)):
    # print_log = open("D:/facedet/"+video_names[i]+'.txt',"w")
    # sys.stdout = print_log
    video_name = video_names[i]
    print('--------------------------------------------')
    print(video_name)
    print('Total frame number: '+str(video_frame_nums[i]))
    print('Blur frame number: '+ str(video_frame_nums[i]-len(video_noblur_frames[i])))
    face_area_sum = 0
    face_num_sum = 0
    human_pixel_ratio = 0
    human_binary_ratio = 0
    min_face_area = 1920*1080
    dance_face_area = []
    camera_face_ratio = []
    dance_thr_num = 0
    with open(os.path.join('/home/bingyu/bingyu/',video_name+'_360p.pkl'), 'rb') as f:
        face_det_result = pickle.load(f)
        for frame_id in range(video_frame_nums[i]):
            if frame_id not in video_noblur_frames[i]:
                humanseg = cv2.imread(os.path.join(root_dir+'humanseg/'+video_name+'.mov/', str(frame_id+1).zfill(8)+'.png'))
                humanseg = cv2.cvtColor(humanseg,cv2.COLOR_RGB2GRAY)
                humanseg = humanseg/255.0
                face_det_i = face_det_result[frame_id]
                # face_num_sum += len(face_det_i[1])
                frame_thr_num = 0
                for j in range(len(face_det_i[1])):
                    det = face_det_i[1][j]
                    det = [deti*bbox_scale for deti in det]
                    det_conf_score = face_det_i[2][j]
                    if det_conf_score > bbox_conf_thr:
                        face_num_sum += 1
                        face_area = (det[3]-det[1])*(det[2]-det[0])
                        face = humanseg[int(det[1]):int(det[3]), int(det[0]):int(det[2])]
                        human_pixel_ratio += np.sum(face)/face_area
                        human_binary_ratio += np.sum((face>0))/face_area
                        face_area_sum += face_area/(1920*1080)
                        if face_area < min_face_area:
                            min_face_area = face_area
                        if frame_id >=2469 and frame_id<=2500:
                            camera_face_ratio += [np.sum(face)/face_area]
                        if frame_id >= 1925 and frame_id <= 1980:
                            dance_face_area += [(det[3]-det[1])*(det[2]-det[0])/(1920*1080)]
                            if (det[3]-det[1])*(det[2]-det[0])/(1920*1080) < 0.008:
                                frame_thr_num = 1
                dance_thr_num += frame_thr_num
    print("Min face area: " + str(min_face_area))
    print("Face number per frame: " + str(face_num_sum/(video_frame_nums[i]-len(video_noblur_frames[i]))))
    print("Average Face area: " + str(face_area_sum/face_num_sum))
    print("Average Face pixel ratio: " + str(human_pixel_ratio/face_num_sum))
    print("Average Face binary pixel ratio: " + str(human_binary_ratio/face_num_sum))
    print(dance_face_area)
    print('Dance face mean area: ' + str(sum(dance_face_area)/len(dance_face_area)))
    print(dance_thr_num)
    print(camera_face_ratio)
    print('Camera face mean ratio: ' + str(sum(camera_face_ratio)/len(camera_face_ratio)))
    # print_log.close()

--------------------------------------------
YRKKH_2800_5min
Total frame number: 7525
Blur frame number: 7297
Min face area: 1360.9422217772808
Face number per frame: 1.3050568726874059
Average Face area: 0.05048994879763025
Average Face pixel ratio: 0.9365381036018642
Average Face binary pixel ratio: 0.9878139810257407
[0.007291983199633654, 0.007511654078659073, 0.0075571395541101085, 0.007478696215063489, 0.007358148781683364, 0.007417097981405176, 0.007614034225908755, 0.007181029742044606, 0.007400831595411243, 0.007273717091949139, 0.0075135985719836955, 0.007269463859073666, 0.007386805887290393, 0.007133023542179823, 0.007465457182676497, 0.007410606474923043, 0.0076020055255958145, 0.0073183003697876205, 0.007294616532439653, 0.007596612165068816, 0.007053353756418801, 0.00785739720635724, 0.007384739677256422, 0.007892314293968007, 0.0072281970426047336, 0.007984797148310463, 0.007301080619769588, 0.00813319912967927, 0.00689710049991542, 0.00825163473421071, 0.00793229338807